# Hyperparameter Tuning Methods Comparison
Cross validation approach based on the following [repo](https://github.com/roamanalytics/roamresearch/tree/master/BlogPosts/Hyperparameter_tuning_comparison)

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%aimport hpt_cmp
%autoreload 1

In [8]:
from hpt_cmp import *

In [3]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from dotmap import DotMap

In [4]:
# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:


In [5]:
param_grid = {
    'C': [1, 10, 100, 1000],  
    'gamma': [0.001, 0.0001],
    'kernel': ['linear', 'rbf']
}

In [9]:
hpt_objs = [
    # add more objs once more search functions implemented
        {
        'name': 'RandomSearch',
        'cv' : random_search,
        'param_grid': param_grid,
        'args': {}
    },
    {
        'name': 'GridSearch',
        'cv': grid_search,
        'param_grid': param_grid,
        'args' : {}
    },
    {
        'name': 'Baseline',
        'cv': run_baseline,
        'param_grid' : {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
        # {}, # use default config
        'args': {}
    }
]

hpt_objs = [DotMap(obj, _dynamic=False) for obj in hpt_objs ]
hpt_objs

[DotMap(name='RandomSearch', cv=<function random_search at 0x7f03f5882e18>, param_grid=DotMap(C=[1, 10, 100, 1000], gamma=[0.001, 0.0001], kernel=['linear', 'rbf']), args=DotMap()),
 DotMap(name='GridSearch', cv=<function grid_search at 0x7f03f5882d90>, param_grid=DotMap(C=[1, 10, 100, 1000], gamma=[0.001, 0.0001], kernel=['linear', 'rbf']), args=DotMap()),
 DotMap(name='Baseline', cv=<function run_baseline at 0x7f03f5882ea0>, param_grid=DotMap(C=1, gamma=0.01, kernel='rbf'), args=DotMap())]

#### cmp_hpt_methods
Parameter description
> `htp_objs`: list of hyperparam-tuning object <br>
> `model`: sklearn model to optimize (needs to have fit/predict function)<br>
> `dataset`: tuple of (X,y) e.g (Data, Target) <br>
> `loss`: sklearn loss function to user <br>
> `metric`: sklearn metric to optimize for <br>
> `datset_split`: random_state for datasetsplit <br>
> `name`: currently not user *optional* <br>

In [10]:
# Run search & print result
results = cmp_hpt_methods(
    hpt_objs,
    SVC,
    (digits.data, digits.target),
    'log_loss',
    accuracy_score)
results

Start
HTP using RandomSearch


RandomSearch
{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}
RandomSearch
{'kernel': 'rbf', 'gamma': 0.001, 'C': 100}

[0.9589345172031076, 0.9665178571428571]
[0.6881961822509766, 0.8627851009368896]
[10, 10]
HTP using GridSearch


GridSearch
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
GridSearch
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

[0.9589345172031076, 0.9665178571428571]
[1.2241437435150146, 1.185934066772461]
[16, 16]
HTP using Baseline


Baseline
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Baseline
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

[0.586015538290788, 0.6785714285714286]
[0.13615083694458008, 0.13722538948059082]
[1, 1]


[{'Model': 'SVC',
  'Hyper optimization method': 'RandomSearch',
  'Test accuracy': [0.9589345172031076, 0.9665178571428571],
  'Best Parameters': [{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000},
   {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}],
  'Parameters sampled': [10, 10],
  'Cross validation time (in s)': [0.6881961822509766, 0.8627851009368896],
  'Mean Test accuracy': 0.9627261871729824,
  'Mean Cross validation time (in s)': 0.7754906415939331,
  'Mean Parameters sampled': 10.0},
 {'Model': 'SVC',
  'Hyper optimization method': 'GridSearch',
  'Test accuracy': [0.9589345172031076, 0.9665178571428571],
  'Best Parameters': [{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'},
   {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}],
  'Parameters sampled': [16, 16],
  'Cross validation time (in s)': [1.2241437435150146, 1.185934066772461],
  'Mean Test accuracy': 0.9627261871729824,
  'Mean Cross validation time (in s)': 1.2050389051437378,
  'Mean Parameters sampled': 16.0},
 {'Model': 'SVC',
  'H